In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, f1_score
from imblearn.under_sampling import RandomUnderSampler
from datetime import timedelta
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

In [119]:
df = pd.read_csv('data/transactions_dataset.csv', sep=';')
clients = pd.read_csv('data/sales_client_relationship_dataset.csv')

# EDA

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
missing_data = df.isnull().sum()

print("Missing values per column:\n", missing_data)

In [ ]:
# Convert 'date_order' and 'date_invoice' to datetime
df['date_order'] = pd.to_datetime(df['date_order'])
df['date_invoice'] = pd.to_datetime(df['date_invoice'])

df['order_channel'].unique()

In [ ]:
print('number of clients in the dataset', len(df['client_id'].unique()))
print('number of stores in the dataset', len(df['branch_id'].unique()))
print('number of type of products in the dataset', len(df['product_id'].unique()))
print('number of channel types in the dataset', len(df['order_channel'].unique()))
print('number of transactions in the dataset', len(df))

In [ ]:
for channel in df['order_channel'].unique() :
    # Filter the dataframe for the current channel
    df_channel = df[df['order_channel'] == channel]

    # Count unique stores in the filtered dataframe
    unique_stores = df_channel['branch_id'].nunique()

    # Print the results
    print(f"Channel: {channel}")
    print(f"revenues per year: {unique_stores}")


# data cleaning and transformaton

In [11]:
# Calculate the delivery time of orders
df['days_to_deliver'] = (df['date_invoice'] - df['date_order']).dt.days
df = df[df.days_to_deliver > 0]

In [ ]:
print(f"there are {df[df.sales_net == 0].shape[0]} rows with sales = 0")
df = df[df.sales_net > 0]

In [ ]:
# Step 1: Transform dataset so that each row is a whole purchase order
df_grouped = df.groupby(["client_id", "date_order"]).agg(
    products_sold_quantities=("product_id", lambda x: dict(zip(x, df.loc[x.index, 'quantity']))),  # Dictionary of product_id: quantity
    branches=("branch_id", lambda x: list(set(x))),  # Unique branches
    sales_channels=("order_channel", lambda x: list(set(x))),  # Unique sales channels
    revenue=('sales_net', sum)
).reset_index()

df_grouped.head()

In [127]:
# Step 2: Add features
df_grouped = df_grouped.sort_values(by=['client_id', 'date_order'])
df_grouped['time_since_last_sale'] = df_grouped.groupby('client_id')['date_order'].diff().dt.days.fillna(0)

# remove outliers and clustering

In [ ]:
# Step 3: Handle outliers (lenient approach)
Q1 = df_grouped['time_since_last_sale'].quantile(0.25)
Q3 = df_grouped['time_since_last_sale'].quantile(0.75)
IQR = Q3 - Q1

# Define the upper bound for outliers
upper_bound = Q3 + 15 * IQR

# Filter the dataframe to remove outliers above the upper bound
df_grouped_filtered = df_grouped[df_grouped['time_since_last_sale'] <= upper_bound]

# Print the number of removed outliers
num_outliers = df_grouped.shape[0] - df_grouped_filtered.shape[0]
print(f"Number of outliers removed: {num_outliers}")
print(f"Which represents {num_outliers / df_grouped.shape[0] * 100:.2f}% of the data points")
print(f"Upper Bound is {upper_bound}")


In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Extract the features for clustering
df_grouped['days_since_last_sale'] = df_grouped['time_since_last_sale']
client_features = df_grouped.groupby('client_id')['days_since_last_sale'].mean().reset_index()
features = client_features[['days_since_last_sale']].values

# Standard scale the features
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Determine the optimal number of clusters using the Elbow Method and Silhouette Score
inertia = []
silhouette_scores = []
K = range(1, 11)

for k in K:
    print(k)
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(features)
    inertia.append(kmeans.inertia_)
    if k > 1:
        silhouette_scores.append(silhouette_score(features, kmeans.labels_))

# Plot the Elbow Method
plt.figure(figsize=(10, 5))
plt.plot(K, inertia, 'bx-')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal k')
plt.show()

# Plot the Silhouette Score
plt.figure(figsize=(10, 5))
plt.plot(K[1:], silhouette_scores, 'bx-')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score For Optimal k')
plt.show()

In [ ]:
# Step 4: Perform clustering
# StandardScaler for scaling
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_grouped_filtered[['time_since_last_sale']])

# KMeans clustering
optimal_k = 4  # Based on prior analysis
kmeans = KMeans(n_clusters=optimal_k, random_state=0)
df_grouped_filtered['client_cluster'] = kmeans.fit_predict(scaled_features)

In [ ]:
# Group by cluster and calculate the number of people in each cluster
cluster_counts = df_grouped_filtered['client_cluster'].value_counts().sort_index()

# Calculate the average, minimum, and maximum time_since_last_sale for each cluster
cluster_stats = df_grouped_filtered.groupby('client_cluster').agg(
    mean_time_since_last_sale=('time_since_last_sale', 'mean'),
    min_time_since_last_sale=('time_since_last_sale', 'min'),
    max_time_since_last_sale=('time_since_last_sale', 'max'),
    number_of_people=('client_id', 'nunique')
)
cluster_stats[['mean_time_since_last_sale', 'number_of_people']]

In [ ]:
# Map clusters to periodicity (based on domain knowledge)
mapping = {
    0: 5,   # Weekly
    1: 30,  # Monthly
    2: 75,  # Quarterly
    3: 150  # Bi-yearly
}
df_grouped_filtered['periodicity'] = df_grouped_filtered['client_cluster'].map(mapping)

df_grouped_filtered.head()

# defining churn

In [ ]:
# Step 5: Define churn based on cluster-specific periodicity
df_grouped_filtered['expected_next_purchase'] = df_grouped_filtered['date_order'] + pd.to_timedelta(df_grouped_filtered['periodicity'], unit='D')
df_grouped_filtered['churn'] = (df_grouped_filtered['expected_next_purchase'] < df_grouped_filtered['date_order'].max()).astype(int)

In [149]:
# Step 6: Feature engineering
# Aggregate client-level features
client_features = df_grouped_filtered.groupby('client_id').agg({
    'time_since_last_sale': 'mean',
    'revenue': 'sum',
    'products_sold_quantities': 'count',  # Number of orders
    'branches': lambda x: len(set([item for sublist in x for item in sublist])),  # Unique branches
    'sales_channels': lambda x: len(set([item for sublist in x for item in sublist])),  # Unique sales channels
    'churn': 'max'  # Target variable
}).reset_index()

#Rename columns for clarity
client_features.rename(columns={
    'time_since_last_sale': 'avg_time_since_last_sale',
    'revenue': 'total_revenue',
    'products_sold_quantities': 'total_orders',
    'branches': 'unique_branches',
    'sales_channels': 'unique_sales_channels'
}, inplace=True)

In [ ]:
client_features

In [151]:
df_grouped_filtered = pd.merge(df_grouped_filtered, clients, on='client_id', how='left')

In [ ]:
df_grouped_filtered

In [137]:
# Step 1: Define the time range for rolling windows
# Get the minimum and maximum order dates from your dataset
min_order_date = df_grouped_filtered['date_order'].min()
max_order_date = df_grouped_filtered['date_order'].max()

# Define the start and end dates for rolling windows
# Example: Use the last 12 months for evaluation, and start 6 months before that
end_date = max_order_date - pd.DateOffset(months=12)  # Last 12 months for testing
start_date = end_date - pd.DateOffset(months=6)       # 6 months before for training

# Step 2: Generate monthly points in time
points_in_time = pd.date_range(start=start_date, end=end_date, freq='MS')  # Monthly intervals

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Step 1: One-Hot Encode categorical columns
def one_hot_encode_categorical(df, categorical_columns):
    """One-hot encode specified categorical columns."""
    encoded_dfs = []
    
    for col in categorical_columns:
        if isinstance(df[col].iloc[0], list):  # Check if the column contains lists
            mlb = MultiLabelBinarizer()
            encoded_data = mlb.fit_transform(df[col])
            encoded_df = pd.DataFrame(encoded_data, columns=[f"{col}_{channel}" for channel in mlb.classes_])
        else:  # Regular categorical column
            encoder = OneHotEncoder(sparse_output=False, drop='first')
            encoded_data = encoder.fit_transform(df[[col]])
            encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out([col]))
        
        encoded_dfs.append(encoded_df)
    
    # Combine encoded data with the original dataframe
    return pd.concat([df.drop(columns=categorical_columns)] + encoded_dfs, axis=1)

# List of categorical columns to encode
categorical_columns = ['quali_relation', 'sales_channels']

# Apply one-hot encoding to the main dataset
df_grouped_filtered_encoded = one_hot_encode_categorical(df_grouped_filtered, categorical_columns)

# Step 2: Update the dataset creation function to include encoded features
def create_datasets_with_periodicity(date, periodicity_multiplier=2):
    # Get periodicity from historical clustering
    periodicity_df = cluster_clients_historical(df_grouped_filtered_encoded, date)
    
    # Filter data up to the given date
    historical_data = df_grouped_filtered_encoded[df_grouped_filtered_encoded['date_order'] < date]
    
    # Aggregate features without future knowledge
    features = historical_data.groupby('client_id').agg(
        recency=('date_order', lambda x: (date - x.max()).days),
        frequency=('date_order', 'count'),
        total_revenue=('revenue', 'sum'),
        unique_branches=('branches', lambda x: x.explode().nunique()),
        **{col: (col, 'sum') for col in df_grouped_filtered_encoded.columns if col.startswith('sales_channels_')},
        **{col: (col, 'sum') for col in df_grouped_filtered_encoded.columns if col.startswith('quali_relation_')}
    ).reset_index()
    
    # Merge periodicity from historical clustering
    features = pd.merge(features, periodicity_df, on='client_id', how='left')
    
    # Define churn: no purchase in periodicity * multiplier days after last date
    features['churn'] = (features['recency'] > features['periodicity'] * periodicity_multiplier).astype(int)
    
    return features

# Step 3: Rebuild datasets with temporal integrity
all_datasets = []
for date in points_in_time:
    dataset = create_datasets_with_periodicity(date, periodicity_multiplier=4)
    all_datasets.append(dataset)
combined_dataset = pd.concat(all_datasets, ignore_index=True)

# Step 4: Time-based train-test split
combined_dataset['max_date'] = pd.to_datetime(points_in_time[-1]) - pd.to_timedelta(combined_dataset['recency'], unit='D')
combined_dataset.sort_values('max_date', inplace=True)

# Split 80-20 temporally
split_idx = int(len(combined_dataset) * 0.8)
X_train = combined_dataset.iloc[:split_idx].drop(columns=['client_id', 'churn', 'max_date'])
y_train = combined_dataset.iloc[:split_idx]['churn']
X_test = combined_dataset.iloc[split_idx:].drop(columns=['client_id', 'churn', 'max_date'])
y_test = combined_dataset.iloc[split_idx:]['churn']

# Step 5: Handle class imbalance (SMOTE instead of undersampling)
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Step 6: Train and evaluate
xgb = XGBClassifier(random_state=42, eval_metric='logloss')
xgb.fit(X_resampled, y_resampled)

# Evaluation
y_pred = xgb.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"AUC: {roc_auc_score(y_test, xgb.predict_proba(X_test)[:,1])}")

# Step 7: Predict churn probabilities for non-churned clients
current_date = df_grouped_filtered_encoded['date_order'].max()
current_data = create_datasets_with_periodicity(current_date, periodicity_multiplier=4)
non_churned = current_data[current_data['churn'] == 0].copy()

# Prepare prediction data (same features as training)
X_live = non_churned.drop(columns=['client_id', 'churn'])

# Generate predictions
non_churned['churn_probability'] = xgb.predict_proba(X_live)[:, 1]

# Sort and format results
churn_risk_list = (non_churned[['client_id', 'churn_probability']]
                   .sort_values('churn_probability', ascending=False)
                   .reset_index(drop=True))

print("Top 10 At-Risk Clients:")
print(churn_risk_list.head(10))

In [ ]:
# Evaluation
y_pred = xgb.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"AUC: {roc_auc_score(y_test, xgb.predict_proba(X_test)[:,1])})")

# Confusion Matrix
confusion = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=confusion)
disp.plot()

plt.title('Confusion Matrix')
plt.show()

print(f"Proportion of churn in dataset {y.mean()*100:.2f} %")

In [ ]:


# Optional: Save results
churn_risk_list.to_csv('client_churn_risk_ranking.csv', index=False)

In [ ]:
churn_risk_list